In [14]:

def read_corners(label_path):
    corners = {}
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            _, u, v, img_name, w, h = line.split(',')
            if img_name not in corners:
                corners[img_name] = []
            corners[img_name].append((int(u), int(v)))

    return corners


corners = read_corners("referenceData/OCamCalib/corner-labels.csv")
print(corners)


{'VMRImage0.jpg': [(678, 372), (706, 375), (740, 380), (777, 383), (816, 390), (859, 396), (861, 442), (814, 429), (775, 419), (735, 410), (701, 401), (671, 395), (661, 419), (691, 431), (726, 443), (765, 458), (807, 473), (852, 493), (649, 446), (678, 461), (711, 477), (750, 498), (791, 519), (836, 544), (633, 472), (660, 489), (691, 513), (726, 536), (766, 564), (808, 594), (614, 495), (638, 518), (665, 543), (695, 573), (732, 601), (768, 636), (597, 517), (617, 540), (639, 568), (664, 599), (694, 632), (726, 667)], 'VMRImage1.jpg': [(640, 331), (655, 324), (674, 311), (701, 295), (735, 274), (777, 245), (803, 295), (752, 315), (712, 329), (684, 340), (661, 346), (643, 354), (649, 376), (668, 373), (690, 370), (721, 367), (765, 361), (818, 355), (821, 413), (767, 410), (725, 406), (695, 401), (669, 400), (651, 397), (653, 422), (671, 426), (694, 432), (722, 441), (761, 452), (809, 469), (793, 512), (750, 489), (716, 473), (690, 459), (670, 448), (653, 441), (653, 459), (668, 469), (6

In [15]:
import numpy as np

class CheckerboardPoint:
    def __init__(self, u, v):
        self.u = u
        self.v = v
        self.x = None
        self.y = None
        self.visited = False

def sort_corners(corners, num_x, num_y):
    # corners: the uv position of the corners on the image. (N, 2)
    # output: the x,y checkerboard index of the corners. (N, 2)

    # 1. find the origin
    centroid = np.mean(corners, axis=0)

    distances_from_centroid = np.linalg.norm(corners - centroid, axis=1)
    farthest_point_idx = np.argmax(distances_from_centroid)
    farthest_point = corners[farthest_point_idx]

    # 2. create a list of CheckerboardPoint
    checkerboard_points = []
    for i, corner in enumerate(corners):
        point = CheckerboardPoint(corner[0], corner[1])
        checkerboard_points.append(point)

    checkerboard_points[farthest_point_idx].x = 0
    checkerboard_points[farthest_point_idx].y = 0
    checkerboard_points[farthest_point_idx].visited = True

    # 3. sort the points cell by cell
    nearest_three_points = sort_adjacent_points(checkerboard_points, checkerboard_points[farthest_point_idx])
    while len(nearest_three_points) > 0:
        new_nearest_three_points = []
        for point in nearest_three_points:
            new_nearest_three_points += sort_adjacent_points(checkerboard_points, point)

        nearest_three_points = new_nearest_three_points

    xy_mat = np.array([[p.x, p.y] for p in checkerboard_points])

    # check if xy are flipped.
    max_x = np.max(xy_mat[:, 0])
    max_y = np.max(xy_mat[:, 1])

    if num_x > num_y and not (max_x > max_y):
        xy_mat = np.flip(xy_mat, axis=1)
    
    print(xy_mat)

def sort_adjacent_points(checkerboard_points, start_point):
    # checkerboard_points: a list of CheckerboardPoint. (N) I should contain the start_point
    if not start_point.visited:
        raise Exception("point is not visited")

    nearest_three_points = find_nearest_three_points(checkerboard_points, start_point)
    
    is_nearest_points_visted = [p.visited for p in nearest_three_points]
    if np.all(is_nearest_points_visted):
        return []

    # find the most ccw point
    angles = []
    for p in nearest_three_points:
        angle = np.arctan2(p.v - start_point.v, p.u - start_point.u)
        angles.append(angle)

    min_angle = min(angles)
    offseted_angles = [angle - min_angle for angle in angles]

    most_ccw_point = nearest_three_points[np.argmax(offseted_angles)]
    most_cw_point = nearest_three_points[np.argmin(offseted_angles)]
    center_point = nearest_three_points[3 - np.argmax(offseted_angles) - np.argmin(offseted_angles)]

    if not most_ccw_point.visited:
        most_ccw_point.x = start_point.x + 1
        most_ccw_point.y = start_point.y
        most_ccw_point.visited = True
    
    if not most_cw_point.visited:
        most_cw_point.x = start_point.x
        most_cw_point.y = start_point.y + 1
        most_cw_point.visited = True
    
    if not center_point.visited:
        center_point.x = start_point.x + 1
        center_point.y = start_point.y + 1
        center_point.visited = True

    # return the nearest points for the next iteration
    return nearest_three_points

def find_nearest_three_points(checkerboard_points, start_point):
    uv_mat = [np.array([p.u, p.v]) for p in checkerboard_points]
    distances = np.linalg.norm(uv_mat - np.array([start_point.u, start_point.v]), axis=1)
    sorted_idx = np.argsort(distances)
    
    nearest_three_idx = sorted_idx[1:4]
    nearest_three_points = [checkerboard_points[idx] for idx in nearest_three_idx]

    return nearest_three_points


num_x = 7
num_y = 6
sample_corners = corners['VMRImage0.jpg']
sample_corners = np.array(sample_corners)

sort_corners(sample_corners, num_x, num_y)


[[10  1]
 [ 9  6]
 [ 8  6]
 [ 7  5]
 [ 6  6]
 [ 5  6]
 [ 4  5]
 [ 5  5]
 [ 6  5]
 [ 7  5]
 [ 8  5]
 [ 9  1]
 [ 8  1]
 [ 7  4]
 [ 6  4]
 [ 5  4]
 [ 4  4]
 [ 3  4]
 [ 7  1]
 [ 6  3]
 [ 5  3]
 [ 4  3]
 [ 3  3]
 [ 2  3]
 [ 6  1]
 [ 5  2]
 [ 4  2]
 [ 3  2]
 [ 2  2]
 [ 1  2]
 [ 5  1]
 [ 4  1]
 [ 3  1]
 [ 2  1]
 [ 1  1]
 [ 0  1]
 [ 5  0]
 [ 4  0]
 [ 3  0]
 [ 2  0]
 [ 1  0]
 [ 0  0]]
